1. I Created database on the Amazon web server, PostgreSQL DB micro instance

In [ ]:
!pip install psycopg2 # allows to connect to PostgreSQL from python
import psycopg2 as ps
from google.colab import drive
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

drive.mount('/drive')
df = pd.read_csv('/drive/My Drive/SyncFolder/projects/API_DB/files/youtube_API_pull.csv', index_col=0)
df

Mounted at /drive


,video_id,video_title,upload_date,view_count,like_count,comment_count
0,mTL23Gd-T3g,Engagement_Ring_Animation,2022-08-26,54,1,0
1,MEnebfZ_CyQ,Ring_and_band_animation,2022-07-29,71,2,1
2,3TxVeAGJXCQ,Ring_animation_diamond,2022-07-27,115,3,1
3,VKVknZOXDSo,white_gold_Ring,2021-03-02,365,13,1
4,cUEueCDG-xY,Ambiguous_Cylinder_Illusion,2020-07-22,127,6,3
5,uasLj9J56os,swiss_blue_topaz_ring,2020-07-10,114,4,3


In [ ]:
# this function allows us to connect to DB
def connect_to_db(host_name, dbname, port, username, password):
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
  return conn

# Create DB table to push date from notbook to DB
#create table with name "videos"
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
          )""")
    curr.execute(create_table_command)

# write insert command to insert into table
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count)
        VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

# sql update row if video exists command
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
              SET video_title = %s,
                  view_count = %s,
                  like_count = %s,
                  comment_count = %s
              WHERE video_id = %s;""")

    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)


#check if video exists function
def check_if_video_exists(curr, video_id):
  query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

  curr.execute(query, (video_id,))
  return curr.fetchone() is not None

def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)

# Update the database
def update_db(curr, df):
    # create pd DataFrame
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date',
                                    'view_count', 'like_count', 'comment_count'])
    # Check to see if video exists
    for i, row in df.iterrows():
          if check_if_video_exists(curr, row['video_id']): #if video already exists then update
                update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count']) #update sql command
          else: # if video doesn't exists append to the db table
                tmp_df = tmp_df.append(row) #insert sql command
    return tmp_df

# Handeling data for scalability while loading df to db
def append_from_df_to_db(curr,df):
    # for loop insert one video at the time 
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])


In [ ]:
#database credentials
host_name = "YourHostName"
dbname = "YourDBName"
port = "5432"
username = "username"
password = "YourPass"
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor() # allows python code to execute SQL commands in database

Connected!


In [ ]:
#create table
create_table(curr)

In [ ]:
# The TRUNCATE TABLE command deletes the data inside a table, but not the table itself.
truncate_table(curr)

In [ ]:
#update data for existing videos
new_vid_df = update_db(curr, df)
new_vid_df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,mTL23Gd-T3g,Engagement_Ring_Animation,2022-08-26,54,1,0
1,MEnebfZ_CyQ,Ring_and_band_animation,2022-07-29,71,2,1
2,3TxVeAGJXCQ,Ring_animation_diamond,2022-07-27,115,3,1
3,VKVknZOXDSo,white_gold_Ring,2021-03-02,365,13,1
4,cUEueCDG-xY,Ambiguous_Cylinder_Illusion,2020-07-22,127,6,3
5,uasLj9J56os,swiss_blue_topaz_ring,2020-07-10,114,4,3


In [ ]:
#insert new videos into db table
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:
#view data in db table
curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('mTL23Gd-T3g', 'Engagement_Ring_Animation', datetime.date(2022, 8, 26), 54, 1, 0), ('MEnebfZ_CyQ', 'Ring_and_band_animation', datetime.date(2022, 7, 29), 71, 2, 1), ('3TxVeAGJXCQ', 'Ring_animation_diamond', datetime.date(2022, 7, 27), 115, 3, 1), ('VKVknZOXDSo', 'white_gold_Ring', datetime.date(2021, 3, 2), 365, 13, 1), ('cUEueCDG-xY', 'Ambiguous_Cylinder_Illusion', datetime.date(2020, 7, 22), 127, 6, 3), ('uasLj9J56os', 'swiss_blue_topaz_ring', datetime.date(2020, 7, 10), 114, 4, 3)]
